# Welcome Message Experiments Meta-Analysis
[J. Nathan Matias](https://twitter.com/natematias), 
June 2021


In addition to CAT Lab's Wikipedia welcome experiment, other communities have also done similar studies. More details are available [here](https://de.wikipedia.org/wiki/Wikipedia:WikiProjekt_Begr%C3%BC%C3%9Fung_von_Neulingen). This notebook attempts a meta-analysis of the two studies.

In [5]:
library(ggplot2)
library(ggpubr)
library(MASS)
library(gmodels)
library(lme4)
library(broom)
library(knitr)
library(lmerTest)
library(texreg)
library(stringr)
library(png)
library(MASS)
library(grid)
library(gridExtra)
library(plm)



### Add Source Sans as a font for ggplot2
library(sysfonts) # to load source sans pro
# https://rdrr.io/github/kjhealy/sourcesans/src/R/sourcesans.r

## Set visual style


cat.logo.filename <- "../../assets/CAT-Logo-Horizontal-social-media-preview-color.png"
logo.img <- readPNG(cat.logo.filename)
logo.pngob <- rasterGrob(logo.img)

Warning message:
“package ‘ggpubr’ was built under R version 3.5.2”
Loading required package: magrittr

Loading required package: Matrix

Warning message:
“package ‘knitr’ was built under R version 3.5.2”

Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Version:  1.36.23
Date:     2017-03-03
Author:   Philip Leifeld (University of Glasgow)

Please cite the JSS article in your publications -- see citation("texreg").


Attaching package: ‘texreg’


The following object is masked from ‘package:magrittr’:

    extract


Warning message:
“package ‘plm’ was built under R version 3.5.2”


## Simulate the EN Teahouse experiment 
The following random assignment was conducted:
 
* control group: 3,092 participants 
  * 247 accounts made at least one edit over 2-3 weeks
* treatment group: 11,674 participants
  * 1068 treatment accounts made at least one edit
  
The measurement period was 3-4 weeks, longer than the FR Wikipedia study but shorter than the DE Wikipedia study. The researchers found a positive, statistically-significant effect only in the 3-4 week period, not in the 1-2 month period or the 2-6 month period. They also did not adjust their results for multiple comparisons. 
  

In [9]:
en.treat.group <- data.frame(TREAT=1, counter=seq(0,11674))
en.treat.group$activation <- as.integer(en.treat.group$counter <1067)
en.control.group <- data.frame(TREAT=0, counter=seq(0,3092))
en.control.group$activation <- as.integer(en.control.group$counter <247)    

en.participants <- rbind(en.treat.group,en.control.group)
en.participants$study <- "EN"

CrossTable(en.participants$TREAT, en.participants$activation, prop.t=FALSE, prop.chisq=FALSE, prop.r=FALSE, prop.c=FALSE)
                                                  


 
   Cell Contents
|-------------------------|
|                       N |
|-------------------------|

 
Total Observations in Table:  14768 

 
                      | en.participants$activation 
en.participants$TREAT |         0 |         1 | Row Total | 
----------------------|-----------|-----------|-----------|
                    0 |      2846 |       247 |      3093 | 
----------------------|-----------|-----------|-----------|
                    1 |     10608 |      1067 |     11675 | 
----------------------|-----------|-----------|-----------|
         Column Total |     13454 |      1314 |     14768 | 
----------------------|-----------|-----------|-----------|

 


In [10]:
summary(lm(activation ~ TREAT, data=en.participants))


Call:
lm(formula = activation ~ TREAT, data = en.participants)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.09139 -0.09139 -0.09139 -0.07986  0.92014 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.079858   0.005119  15.600   <2e-16 ***
TREAT       0.011534   0.005757   2.003   0.0452 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2847 on 14766 degrees of freedom
Multiple R-squared:  0.0002717,	Adjusted R-squared:  0.000204 
F-statistic: 4.014 on 1 and 14766 DF,  p-value: 0.04515


## Simulate the DE experiment
The following random assignment was conducted:
* Treatment group received no welcome message (N=7209)
* Control group received an automated welcome message, with a signature from a rotating cast of experienced, self-selected accounts (N=7329)

Observations were made over a 56 day period (much longer than our 7 day activation period), and the system recorded whether they made at least one edit in that period.

In [11]:
de.treat.group            <- data.frame(TREAT=1, counter=seq(0,7209))
de.treat.group$activation <- as.integer(de.treat.group$counter < 683)
summary(factor(de.treat.group$activation))

de.control.group            <- data.frame(TREAT=0, counter=seq(0,7329))
de.control.group$activation <- as.integer(de.control.group$counter < 699)

de.participants <- rbind(de.treat.group, de.control.group)
de.participants$study <- "DE"

CrossTable(de.participants$TREAT, de.participants$activation, prop.t=FALSE, prop.chisq=FALSE, prop.r=FALSE, prop.c=FALSE)

0    1 
6527  683


 
   Cell Contents
|-------------------------|
|                       N |
|-------------------------|

 
Total Observations in Table:  14540 

 
                      | de.participants$activation 
de.participants$TREAT |         0 |         1 | Row Total | 
----------------------|-----------|-----------|-----------|
                    0 |      6631 |       699 |      7330 | 
----------------------|-----------|-----------|-----------|
                    1 |      6527 |       683 |      7210 | 
----------------------|-----------|-----------|-----------|
         Column Total |     13158 |      1382 |     14540 | 
----------------------|-----------|-----------|-----------|

 


In [12]:
summary(lm(activation ~ TREAT, data=de.participants))


Call:
lm(formula = activation ~ TREAT, data = de.participants)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.09536 -0.09536 -0.09473 -0.09473  0.90527 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.095362   0.003426   27.84   <2e-16 ***
TREAT       -0.000632   0.004865   -0.13    0.897    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2933 on 14538 degrees of freedom
Multiple R-squared:  1.161e-06,	Adjusted R-squared:  -6.762e-05 
F-statistic: 0.01688 on 1 and 14538 DF,  p-value: 0.8966


## Load the FR Experiment Data

In [13]:
data.dir = "~/Tresors/CivilServant/projects/wikipedia-integration/fr-newcomer-study/datasets/post-experiment/"
filename = "frwiki-welcome-post-treatment-vars.csv"

## French Participants
participants <- read.csv(paste(data.dir, filename, sep=""),  na.string = "" )

### Identify and Remove Incomplete Blocks

In [14]:
participants$counter <- 1
block.sizes <- aggregate(participants[c("counter")], by=list(participants$randomization.block.id), FUN=sum)
incomplete.blocks <- subset(block.sizes, counter!=6)$Group.1
print(paste("Removing the following block for being incomplete:", incomplete.blocks))
removed.participants <- subset(participants, (randomization.block.id %in% c(incomplete.blocks)))
participants <- subset(participants, (randomization.block.id %in% c(incomplete.blocks))!=TRUE)

[1] "Removing the following block for being incomplete: 1"   
[2] "Removing the following block for being incomplete: 9516"


In [15]:
participants$control = ifelse(participants$randomization.arm == 0, 1, 0)
participants$treat.one = ifelse(participants$randomization.arm == 1, 1, 0)
participants$treat.two = ifelse(participants$randomization.arm == 2, 1, 0)

## for this meta-analysis, we care about whether someone received any welcome or not
participants$TREAT <- participants$randomization.arm > 0 

## setting the same variable as the DE study
participants$activation = ifelse(participants$seven.day.activation == "True", 1, 0)
participants$study = "FR"

## Merge FR and DE and EN Data

In [16]:
all.participants <- rbind(participants[c('TREAT', 'activation', 'study')], 
                          de.participants[c("TREAT", 'activation', 'study')],
                          en.participants[c("TREAT", 'activation', 'study')])

In [17]:
CrossTable(all.participants$TREAT, all.participants$study, prop.t=FALSE, prop.chisq=FALSE, prop.r=FALSE, prop.c=FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|-------------------------|

 
Total Observations in Table:  86392 

 
                       | all.participants$study 
all.participants$TREAT |        DE |        EN |        FR | Row Total | 
-----------------------|-----------|-----------|-----------|-----------|
                     0 |      7330 |      3093 |     19028 |     29451 | 
-----------------------|-----------|-----------|-----------|-----------|
                     1 |      7210 |     11675 |     38056 |     56941 | 
-----------------------|-----------|-----------|-----------|-----------|
          Column Total |     14540 |     14768 |     57084 |     86392 | 
-----------------------|-----------|-----------|-----------|-----------|

 


In [18]:
## FR model
summary(fr.1 <- lm(activation ~ TREAT, data=participants))

## DE model
summary(de.1 <- lm(activation ~ TREAT, data=de.participants))

## en model
summary(en.1 <- lm(activation ~ TREAT, data=de.participants))


## combined model
summary(re.1 <- lmer(activation ~ TREAT + (1|study), data=all.participants))


Call:
lm(formula = activation ~ TREAT, data = participants)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.3707 -0.3707 -0.3686  0.6293  0.6314 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.368562   0.003500 105.299   <2e-16 ***
TREATTRUE   0.002181   0.004287   0.509    0.611    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4828 on 57082 degrees of freedom
Multiple R-squared:  4.535e-06,	Adjusted R-squared:  -1.298e-05 
F-statistic: 0.2589 on 1 and 57082 DF,  p-value: 0.6109



Call:
lm(formula = activation ~ TREAT, data = de.participants)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.09536 -0.09536 -0.09473 -0.09473  0.90527 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.095362   0.003426   27.84   <2e-16 ***
TREAT       -0.000632   0.004865   -0.13    0.897    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2933 on 14538 degrees of freedom
Multiple R-squared:  1.161e-06,	Adjusted R-squared:  -6.762e-05 
F-statistic: 0.01688 on 1 and 14538 DF,  p-value: 0.8966



Call:
lm(formula = activation ~ TREAT, data = de.participants)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.09536 -0.09536 -0.09473 -0.09473  0.90527 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.095362   0.003426   27.84   <2e-16 ***
TREAT       -0.000632   0.004865   -0.13    0.897    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2933 on 14538 degrees of freedom
Multiple R-squared:  1.161e-06,	Adjusted R-squared:  -6.762e-05 
F-statistic: 0.01688 on 1 and 14538 DF,  p-value: 0.8966


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: activation ~ TREAT + (1 | study)
   Data: all.participants

REML criterion at convergence: 98187.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.8686 -0.8686 -0.2194  1.4730  2.1385 

Random effects:
 Groups   Name        Variance Std.Dev.
 study    (Intercept) 0.02576  0.1605  
 Residual             0.18236  0.4270  
Number of obs: 86392, groups:  study, 3

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)
(Intercept) 1.828e-01  9.270e-02 2.002e+00   1.972    0.187
TREAT       2.856e-03  3.117e-03 8.639e+04   0.916    0.360

Correlation of Fixed Effects:
      (Intr)
TREAT -0.022